# 実験概要
## CMNによる特徴量抽出
**ケプストラム平均正規化** (Cepstrum Mean Normalization : CMN) を使用し、特徴量を抽出する。
切り出した波形に対して**離散フーリエ変換** (Discrete Fourier Transform : DFT) を行い、絶対値を取ることで**振幅スペクトル**を得る。
これに対して対数を取って**対数振幅スペクトル**に変換し、逆離散フーリエ変換 (Inverse Discrete Fourier Transform : IDFT) を行うことで、ケプストラム領域へと変換し、先頭50要素を切り出し、結合した100要素の配列を前処理として返す。
この前処理は
```
left, right, posture = pp.slicer("raw\\" + tester.**.**.value)
cepstrum = pp.cmn_denoise(left, right)
```
によって行われる。

## DNNによる寝姿勢分類
DNNによって前処理したデータから学習・分類を行う。

# モジュールのインポート

In [1]:
import numpy as np
import glob

# 自作モジュール
import datapath as dpath
import preprocess as pp

# DataLoaderの定義

In [2]:
# tester以外のrawを読み込む
for p in glob.glob(".\\raw\\LMH\\*\\", recursive=True):
    print(p)

In [3]:
e = dpath.LMH.H002.value.fl_center
print(e.value)

type, tester, mattress = dpath.getattributes(e, position=False)
print(f"type : {type},  tester : {tester},  mattress : {mattress}")
train_paths = eval(f"dpath.{type}.serch('{mattress}')")
print(train_paths)

raw\LMH\H002\fl_center
type : LMH,  tester : H002,  mattress : fl
[WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M002/fld_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M003/fld_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M004/fls_center'), WindowsPath('raw/LMH/M004/fld_center'), WindowsPath('raw/LMH/M004/fls_center'), WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/

In [15]:
import torch
from typing import List, Tuple
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 学習用データセットを読み込むためのDataLoaderを定義
class train_datasets(Dataset):
    def __init__(self, identity, transforms=None):
        self.transform = transforms
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print(self.type, self.tester, self.mattress)
        print("train")

        # tester以外のrawを読み込む
        self.train_cepstrum = np.empty((0, 100))  # 100要素の配列
        self.train_posture = np.empty(0) # 姿勢データ配列

        # mattressに該当するrawのpathを読み込む
        train_paths = eval(f"dpath.{self.type}.serch('{self.mattress}')")
        for p in train_paths:
            if identity.value == p:
                continue
            left, right, posture = pp.slicer(p)
            cepstrum = pp.cmn_denoise(left, right)
            self.train_cepstrum = np.append(self.train_cepstrum, cepstrum[:50], axis=0)
            self.train_posture = np.append(self.train_posture, posture)

    def __len__(self):
        return len(self.train_posture)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        return self.train_cepstrum[idx], self.train_posture[idx]

# テスト用データセットを読み込むためのDataLoaderを定義
class test_datasets(Dataset):
    def __init__(self, identity, transform=None):
        self.transform = transform
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print("test")

        # 前処理したrawを読み込む
        self.left, self.right, self.posture = pp.slicer(identity.value)
        self.cepstrum = pp.cmn_denoise(self.left, self.right)

    def __len__(self):
        return len(self.cepstrum)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        return self.cepstrum[idx], self.posture[idx]

# 学習＆検証

In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## モデル定義

In [16]:
import torch.nn as nn

# 各層のノード数を定義
n_input = 100
n_output = 4
n_hidden = 128

# 多層NNの定義
class Net(nn.Module):
    def __init__(self, n_input, n_output, n_hidden):
        super().__init__()
        self.layer1 = nn.Linear(n_input, n_hidden)
        self.layer2 = nn.Linear(n_hidden, n_hidden)
        self.layer3 = nn.Linear(n_hidden, n_output)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [17]:
identity = dpath.LMH.H002.value.st_center

train = train_datasets(identity)
test = test_datasets(identity)

train_loader = DataLoader(
    train,
    batch_size=32,
    shuffle=True,
)

test_loader = DataLoader(
    test,
    batch_size=32,
    shuffle=True,
)

LMH H002 st
train
data[340 / 403]


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 0 and the array at index 1 has size 50

## 学習

In [8]:
import torch.optim as optim

# モデルのインスタンス化
net = Net(n_input, n_output, n_hidden).to(device)

# 誤差関数を交差エントロピーで計算
criterion = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters(), lr = 0.1)

# 学習
n_epoch = 100
for epoch in range(n_epoch):
    # 精度と損失の初期化
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    n_train, n_test = 0, 0

    # 学習
    for input, label in train_loader:
        n_train += len(label)

        # 入力と正解ラベルをGPU上に移動
        input = input.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        predicted = torch.max(output, 1)[1]

        train_loss += loss.item()
        train_acc += (predicted == label).sum().item()

    # 検証
    for test_input, test_label in test_loader:
        n_test += 1

        test_input = test_input.to(device)
        test_label = test_label.to(device)

        test_output = net(test_input)
        test_loss = criterion(test_output, test_label)

        test_predicted = torch.max(test_output, 1)[1]

        test_loss += test_loss.item()
        val_acc += (test_predicted == test_label).sum().item()

    # 精度を確率に変換
    train_acc /= n_train
    val_acc /= n_test

    print(f"Epoch[{epoch+1}/{n_epoch}], | loss: {train_loss:.5f} | acc: {train_acc:.5f} | val_loss: {val_loss:.5f} | val_acc: {val_acc:.5f}")
    items = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc])
    history = np.vstack((history, items))


IndexError: index 2570 is out of bounds for axis 0 with size 700